In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import random
import seaborn as sns
from IPython.display import display
import matplotlib.pyplot as plt
# from matplotlib import animation
# from JSAnimation.IPython_display import display_animation
# import gym
import scipy.stats as stats
from scipy.stats import norm
from collections import namedtuple
import statistics
import time
import os
from collections import deque
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque

import yfinance as yf
import datetime as dt

import pickle

/Users/weihangoh/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/weihangoh/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
/Users/weihangoh/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:

class Environment:
    def __init__(self, stock_data, option_data, features_data, T, n_steps, num_sold_opt, kappa, alpha=0, gamma=0.99):
        self.stock_data = stock_data
        self.option_data = option_data
        self.features_data = features_data
        self.T = T
        self.n_steps = n_steps
        self.num_sold_opt = num_sold_opt
        self.kappa = kappa
        self.alpha = alpha
        self.gamma = gamma
        self.dt = T / n_steps
        self.state = None
        self.idx_time = 0
        self.initial_idx_time = 0
        self.min_holdings = -100    # Short selling up to 100 stocks
        self.max_holdings = 100     # Long position up to 100 stocks

        
         # Adjust the number of states based on whether features are provided
        if self.features_data is not None:
            self.num_states = 5 + self.features_data.shape[1]  # Time to expiration, Stock Price, Option Price, Delta + number of features
        else:
            self.num_states = 5  # Only core state variables: T, S, delta, num_stk, gamma
            
        self.num_actions = 1
        
    def reset(self):
        # Choose the length based on whether features_data is provided
        data_length = len(self.features_data) if self.features_data is not None else len(self.stock_data)
        # Select a random starting point, ensuring there's enough room for the episode
        self.initial_idx_time = random.randint(0, data_length - self.n_steps - 1)
        # self.initial_idx_time = random.randint(0, len(self.stock_data) - self.n_steps - 1) --> not using this bc features may have fewer rows if drop nan
        self.idx_time = self.initial_idx_time
        self.current_max_steps = self.idx_time + self.n_steps
        self.state = self._get_state()
        return self.state

    def step(self, action):
        #Take action in the environment
        if self.idx_time >= self.n_steps + self.initial_idx_time:
            # Terminal state: no next state
            reward = self._calculate_reward(terminal=True)
            pnl_value = self._calculate_pnl(None)
            return None, reward, True, pnl_value
        
        nS0 = action  # Number of stocks held at the current time step
         # Extract core state variables (T, S, delta, num_stk)
        T0, S0, delta0, num_stk0 = self.state[:4]
        # The rest are features, which can be used as needed
        features0 = self.state[4:]
        
        # Move to the next time step
        self.idx_time += 1
        next_state = self._get_state()
        # Extract core state variables for the next state
        T1, S1, delta1, nS1 = next_state[:4]
        features1 = next_state[4:]
        
        # Retrieve daily volatility from option_data for reward calculation
        # vol = self.option_data['impliedVolatility'].iloc[self.idx_time % len(self.option_data)] 
        idx = self.idx_time % len(self.option_data)
        vol = self.option_data['impliedVolatility'].iloc[idx]
        
        C0 = self.option_data['OptionPrice'].iloc[idx - 1]
        C1 = self.option_data['OptionPrice'].iloc[idx]

        # Reward calculation
        reward = self._calculate_reward(T0, T1, S0, S1, delta0, delta1, nS0, nS1, vol, action, C0, C1, terminal=False)
        done = (self.idx_time >= self.current_max_steps)
        pnl_value = self._calculate_pnl(next_state)

        self.state = next_state
        return self._get_state(), reward, done, pnl_value

    def _get_risk_free_rate(self):
        """
        Helper function to retrieve the risk-free rate based on data availability.
        """
        if self.features_data is not None:
            # Check if 'RiskFreeRate' is in a multi-index format
            if isinstance(self.features_data.columns, pd.MultiIndex):
                return self.features_data.loc[self.stock_data.index[self.idx_time], ('RiskFreeRate', '')]
            else:
                return self.features_data.loc[self.stock_data.index[self.idx_time], 'RiskFreeRate']
        else:
            # Fallback to stock_data if features_data is unavailable
            return self.stock_data['RiskFreeRate'].iloc[self.idx_time]
        
    def _get_state(self):
        """
        Retrieves the current state.
        
        Returns:
        - A numpy array representing the state.
        """
        S = self.stock_data['Adj Close'].iloc[self.idx_time]
        option_row = self.option_data.iloc[self.idx_time % len(self.option_data)]
        #(f"n_steps: {self.n_steps}, idx_time: {self.idx_time}")
        T = (self.current_max_steps - self.idx_time)
        K = option_row['strike']
        sigma = option_row['impliedVolatility']  # daily volatility
        r = self._get_risk_free_rate()  # Get risk-free rate using helper
        
        delta = self._calculate_delta(S, K, T, r, sigma)
        gamma = self.calculate_gamma(S, K, T, r, sigma)
        num_stk = 0
    
        # Check if features are provided
        if self.features_data is not None:
            features = self.features_data.iloc[self.idx_time]
            return np.array([T, S, delta, num_stk, gamma] + list(features))
        else:
            return np.array([T, S, delta, num_stk, gamma])
    
    def _calculate_delta(self, S, K, T, r, sigma):
        """
        Black-Scholes formula to calculate delta of a call option.
        
        Parameters:
        - S: Stock price.
        - K: Strike price.
        - T: Time to expiration (in years).
        - r: Risk-free interest rate.
        - sigma: Volatility of the underlying stock.
        
        Returns:
        - delta: The calculated delta of the call option.
        """
        #print(f"S: {S}, K: {K}, T: {T}, r: {r}")
        if T <= 0:
            return 0
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        delta = norm.cdf(d1)
        return delta

    def calculate_gamma(self, S, K, T, r, sigma):
        """
        Calculates gamma using the Black-Scholes formula.
        
        Parameters:
        - S: Current stock price.
        - K: Strike price of the option.
        - T: Time to expiration in years.
        - r: Risk-free interest rate.
        - sigma: Volatility of the underlying stock.
        
        Returns:
        - gamma: The gamma of the option.
        """
        if T <= 0 or sigma <= 0:
            return 0

        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
        return gamma

    def _calculate_reward(
        self, T0=None, T1=None, S0=None, S1=None, delta0=None, delta1=None,
        nS0=None, nS1=None, vol=None, action=None, C0=None, C1=None, terminal=False
    ):
        if terminal:
            # Terminal reward calculation
            initial_option_price = self.option_data['OptionPrice'].iloc[self.initial_idx_time]
            r = self.num_sold_opt * (np.exp(self._get_risk_free_rate() * self.T) - 1) * initial_option_price
            r = r / np.power(self.gamma, self.n_steps - self.T)
            return r
        
        # Check for NaN values in inputs
        if any(np.isnan([S0, S1, delta0, delta1, nS0, nS1, vol, action, C0, C1])):
            print(f"NaN detected in reward calculation inputs: S0={S0}, S1={S1}, delta0={delta0}, delta1={delta1}, nS0={nS0}, nS1={nS1}, vol={vol}, action={action}, C0={C0}, C1={C1}")
            return 0.0  # Or handle as per your logic

        # Ensure holdings are within allowed limits
        nS1 = np.clip(nS1, self.min_holdings, self.max_holdings)

        # Proceed with reward calculation
        stock_position_reward = nS1 * S1 - nS0 * S0
        option_position_reward = -self.num_sold_opt * (C1 - C0)
        transaction_cost = self.alpha * S0 * abs(nS1 - nS0) + 0.01 * (nS1 - nS0) ** 2
        risk_cost = 0
        if self.kappa > 0:
            risk_cost = self.kappa * (vol * S0 * (nS1 - delta0)) ** 2 * self.dt / 2

        reward = stock_position_reward - option_position_reward - transaction_cost - risk_cost

        # Apply the gamma discount factor
        reward = reward / np.power(self.gamma, self.n_steps -  T0)

        # Check for NaN in the resulting reward
        if np.isnan(reward):
            print(f"NaN reward detected: stock_position_reward={stock_position_reward}, option_position_reward={option_position_reward}, "
                f"transaction_cost={transaction_cost}, risk_cost={risk_cost}")

        return reward

    def _calculate_pnl(self, n_state):
        # Extract real-world data based on the current time index
        if self.idx_time >= self.initial_idx_time + len(self.stock_data):
            # If no next state or if we are at the end, compute terminal value
            current_stock_price = self.stock_data['Adj Close'].iloc[self.idx_time]
            current_option_value = max(current_stock_price - self.option_data['strike'].iloc[0], 0)  # Assuming a call option payoff at expiration
            pnl = self.num_sold_opt * (np.exp(self.T) - 1) * current_option_value
            return pnl

        # Extract the relevant time information and market data
        t1 = n_state[0]
        t0 = self.state[0]

        # Stock prices at times t0 and t1 from real-world data
        S0 = self.stock_data['Adj Close'].iloc[self.idx_time - 1]  # Stock price at time t0
        S1 = self.stock_data['Adj Close'].iloc[self.idx_time]      # Stock price at time t1

        # Option parameters
        option_row_0 = self.option_data.iloc[(self.idx_time - 1) % len(self.option_data)]
        option_row_1 = self.option_data.iloc[self.idx_time % len(self.option_data)]
        K0 = option_row_0['strike']
        K1 = option_row_1['strike']
        sigma_0 = option_row_0['impliedVolatility']
        sigma_1 = option_row_1['impliedVolatility']
        risk_free_rate = self._get_risk_free_rate()  # Get risk-free rate using helper

        # Calculate delta of the option at times t0 and t1
        d0 = self._calculate_delta(S=S0, K=K0, T=t0, r=risk_free_rate, sigma=sigma_0) * self.num_sold_opt
        d1 = self._calculate_delta(S=S1, K=K1, T=t1, r=risk_free_rate, sigma=sigma_1) * self.num_sold_opt

        # Number of stocks held by the agent at times t0 and t1
        nS0 = self.state[3]  # Number of stocks at t0
        nS1 = n_state[3]     # Number of stocks at t1

        # Calculate the portfolio value components
        pnl = nS1 * S1 - nS0 * S0  # Profit from stock holdings
        pnl -= self.num_sold_opt * (S1 - S0)  # Change in option value
        pnl -= (nS1 - nS0) * S0 * np.exp(risk_free_rate * (self.T - t0))  # Cost for changing stock positions

        # Calculate transaction costs if applicable
        if self.alpha > 0:
            cost = self.__get_cost(S=S0, chg_nS=(nS1 - nS0))
            pnl -= cost * np.exp(risk_free_rate * (self.T - t0))

        return pnl

    def __get_cost(self, S, chg_nS):
        return self.alpha * S * (np.abs(chg_nS) + 0.01 * chg_nS**2)
